Gradient Boosting uses cycles to iteratively add models to an ensemble.

- Initialized with a single model whose predictions can be inaccurate.
- Predictions are used to calculate a loss function.
- The function is used to fit the next new model in the ensemble by choosing parameters that will reduce the loss.
- Predictions are calculated using the ensemble.
- Cycle continues...

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('melb_data.csv')

cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

y = data.Price

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [5]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [6]:
from sklearn.metrics import mean_absolute_error

predictions = model.predict(X_valid)
print(mean_absolute_error(predictions, y_valid))

237479.72280007365


### Parameter Tuning
Essential step because the right parameters can significantly increase performance.

`n_estimators`: 
- How many times to go through the modeling cycle. 
- Equal to the number of models to include in the ensemble.
- Too high -> Causes overfitting, Too low -> Causes underfitting.

`early_stopping_rounds`:
- Offers a way to automatically find the ideal value for `n_estimators`.
- Indicates how many rounds with deteriorating results to allow before stopping.
- 5 is a reasonable value.

`eval_set`:
- Needed parameter when using `early_stopping_rounds`.
- Points to the data needed for calculating the validation scores.

`learning_rate`:
- Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in.
- This decreases the weight of each tree, meaning that we can set a higher value for `n_estimators` without overfitting.
- **Generally, small learning rate and high number of estimators lead to higher accuracy models.

`n_jobs`:
- Used when working with larger dataset to apply parallelism to build models faster.
- Commonly set to the number of cores in one's machine.
- Does not help on smaller datasets.

In [52]:
my_model = XGBRegressor(n_estimators=900, learning_rate=0.05, n_jobs=4, early_stopping_rounds=100)
my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,100
,enable_categorical,False
,eval_metric,None


In [53]:
preds = my_model.predict(X_valid)
print(mean_absolute_error(preds, y_valid))
print(my_model.best_iteration)


234854.10259112666
573
